In [11]:
# 안정적으로 LLM을 활용하기 위해서는 사용자의 요청 둥 적절하지 않은 요청(예: 시스템 프롬프트 요청, 정치적 입장을 묻는 질문)에는 응답하지 않고 벡터 데이터베이스 내 검색 결과나 LLM의 생성 결과에 적절하지 않은 내용(예: 민감한 개인 정보)이 포함됐는지 확인하는 절차가 필요하다.
# 벡터 데이터베이스에서 검색한 데이터나 LLM이 생성한 데이터에는 개인 정보나 서비스에 부적절한 데이터가 섞여 있을 수 있다.
# 특히, LLM이 생성한 데이터의 경우 필요한 형힉에 맞는지(예:JSON), 회사의 정책이나 가이드라인과 충돌하는 내용은 없는지 확인이 필요하다.
# 이때 사용할 수 있는 방법은 1) 규칙 기반, 2) 분류 또는 회귀 모델, 3) 임베딩 유사도 기반, 4) LLM 활용

In [ ]:
import os
from nemoguardrails import LLMRails, RailsConfig
import nest_asyncio

nest_asyncio.apply()

os.environ["OPENAI_API_KEY"] = "KEY"

In [13]:
colang_content = """
define user greeting
  "안녕!"
  "How are you?"
  "What's up?"

define bot express greeting
  "안녕하세요!"

define bot offer to help
  "어떤 것을 도와드릴까요?"

define flow
  user  greeting
  bot express greeting
  bot offer to help
""" # 사용자의 요청을 정의하고 정의한 사용자 요청과 유사한 요청이 들어왔을 때 어떤 응답을 생성할지 미리 정의할 수 있다.

In [14]:
yaml_content = """
models:
  - type: main
    engine: openai
    model: gpt-3.5-turbo

  - type: embeddings
    engine: openai
    model: text-embedding-ada-002
"""

In [15]:
config = RailsConfig.from_content(
    colang_content=colang_content,
    yaml_content=yaml_content
)

In [16]:
rails = LLMRails(config=config)

In [17]:
rails.generate(messages=[{"role": "user", "content": "안녕하세요!"}])

{'role': 'assistant', 'content': '안녕하세요!'}

In [18]:
# 특정 분야(예: 요리)에 대한 질문이나 요청에 답변하지 않도록 하고 싶은 경우
colang_content_cooking = """
define user ask about cooking
  "How can I cook pasta?"
  "How much do I have to boil pasta?"
  "파스타 만드는 법을 알려줘."
  "요리하는 방법을 알려줘."

define bot refuse to respond about cooking
  "죄송합니다. 저는 요리에 대한 정보는 답변할 수 없습니다. 다른 질문을 해주세요."

define flow cooking
  user ask about cooking
  bot refuse to respond about cooking
"""

In [19]:
config = RailsConfig.from_content(
    colang_content=colang_content_cooking,
    yaml_content=yaml_content
)

In [20]:
rails_cooking = LLMRails(config=config)

In [21]:
rails_cooking.generate(messages=[{"role": "user", "content":"사과 파이는 어떻게 만들어?"}])

{'role': 'assistant',
 'content': '죄송합니다. 저는 요리에 대한 정보는 답변할 수 없습니다. 다른 질문을 해주세요.'}